In [126]:
# Setup Spark Session
# Settion Session Name: Assingment07
# Printing Spark Version
from pyspark.sql import SparkSession

# Get Imports Needed
from pyspark.sql.functions import lower, col, udf, lit

# Get Datatypes needed for DataFrame manipulation
from pyspark.sql.types import IntegerType, StringType

# Setup Spark Session
sc = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("Assignment07") \
        .getOrCreate()

# Print Spark Version being Run
print("Spark V: ", sc.version)

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
# Get the Sentiment Value (0/1) from the line and return as an Integer
def GetSentiment(line):
    res = line.strip()[-1]
    return int(res)
get_sentiment = udf(lambda q : GetSentiment(q), IntegerType())

# Get the Sentence by removing the last charater and then cleaning it up
# Returing the sentence as lower case string
def GetSentence(line):
    res = line.strip()[:-1].strip()
    return res.lower()
get_sentence = udf(lambda q : GetSentence(q), StringType())

In [ ]:
# Read the three (3) dataset files into dataframes
# Adding the "source" to each dataframe
df1 = sc.read.text("datasets/amazon_cells_labelled.txt").withColumn("source", lit("amazon"))
df2 = sc.read.text("datasets/imdb_labelled.txt").withColumn("source", lit("imdb"))
df3 = sc.read.text("datasets/yelp_labelled.txt").withColumn("source", lit("yelp"))

# Concatenate the dataframe df1, df2, and df3 with eachoather to form a "df"
df = df1.union(df2).union(df3)

# Cleanup Data and set two columns (sentence & sentiment) int eh dataframe
df = df.withColumn("sentence", get_sentence(col("value")))
df = df.withColumn("sentiment", get_sentiment(col("value")))

# remove the inital "value" column from the dataframe
df = df.drop("value")
df.printSchema()
df.count()

df1 = df.randomSplit([0.8, 0.2], 16)


In [125]:
# Split Dataframe for Trainign and Testing
# I opted for a 80 / 20 Split
# Using the Random Split I get two arrays ech with their respective number of items
data = df.randomSplit([0.8, 0.2], 16)


ConnectionRefusedError: [Errno 61] Connection refused